In [10]:
from path_arquivos import *
import pandas as pd
import numpy as np

In [11]:

df_96 = pd.read_excel(ar_xlsx.ar_96, usecols= ['CODPROD', 'DESCRICAO', 'OBS2', 'QTUNITCX', 'LASTROPAL', 'ALTURAPAL', 'QTTOTPAL','ALTURAARM', 'LARGURAARM', 'COMPRIMENTOARM', 'CAPACIDADE', 'ABASTECEPALETE', 'RUA', 'PREDIO', 'NIVEL', 'APTO','PRAZOVAL'])
colunas_int = ['CODPROD','QTUNITCX', 'LASTROPAL', 'QTTOTPAL', 'ALTURAPAL', 'CAPACIDADE','PRAZOVAL']
colunas_float = [ 'ALTURAARM', 'LARGURAARM', 'COMPRIMENTOARM']
df_96[colunas_int] = df_96[colunas_int].fillna(0).astype(int)  
df_96[colunas_float] = df_96[colunas_float].fillna(0).astype(float) 
df = df_96[df_96["RUA"].between(1,39)].copy()

c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
antigo = ['CODPROD', 'DESCRICAO', 'ABASTECEPALETE', ]
novo = ['COD', 'DESC', 'FLEG_ABST']
dic_zip = dict(zip(antigo, novo))
df.rename(columns=dic_zip, inplace= True)
df['FLEG_VAL'] = np.where(df['PRAZOVAL'] == 0,"NÃO","SIM")
display(df)

,COD,DESC,OBS2,QTUNITCX,LASTROPAL,ALTURAPAL,QTTOTPAL,ALTURAARM,LARGURAARM,COMPRIMENTOARM,CAPACIDADE,FLEG_ABST,RUA,PREDIO,NIVEL,APTO,PRAZOVAL,FLEG_VAL
19,473528,ABRACADEIRA NYLON BRANCA 2.5MM X 100MM,NaN,50,15,5,75,13.0,20.5,30.0,15,NÃO,32,1,1,903,720,SIM
20,473536,ABRACADEIRA NYLON PRETA 2.5MM X 100MM,NaN,50,15,5,75,13.5,21.0,39.5,15,NÃO,32,9,1,904,0,NÃO
26,468976,ABS ADULTO BIGFRAL PLUS S/AB UNISSEX20UN,NaN,8,6,5,30,19.5,42.0,45.0,12,NÃO,19,23,1,102,730,SIM
27,468063,ABS ADULTO LIFREE CALCA 20UN,NaN,1,8,5,40,27.0,39.0,40.0,15,NÃO,20,52,1,103,730,SIM
28,449481,ABS ALWAYS NOT SECA 28CM C/AB G L16P14,NaN,12,18,3,54,34.0,20.0,32.5,15,NÃO,19,29,1,103,730,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14148,466533,XAROPE MONIN LE FRUIT 1LT KIWI,NaN,4,30,4,120,33.0,18.0,18.0,60,NÃO,7,48,1,104,720,SIM
14149,466534,XAROPE MONIN LE FRUIT 1LT MARACUJA,NaN,4,30,4,120,33.0,18.0,18.0,20,NÃO,7,78,1,104,720,SIM
14150,456261,XAROPE MONIN LE FRUIT 1LT MIRTILO,NaN,4,30,4,120,33.0,18.0,18.0,20,NÃO,7,59,1,101,720,SIM
14151,456260,XAROPE MONIN LE FRUIT 1LT MORANGO,NaN,4,30,4,120,33.0,18.0,18.0,15,NÃO,7,88,1,203,720,SIM
